In [1]:
""" ---------------------------------------------------------------------------------------------------------------------------
# Test script to establish AI.zymes on SCC
" -----------------------------------------------------------------------------------------------------------------------------
""" 

import sys, os
if os.path.join(os.getcwd(), '../../src') not in sys.path: sys.path.append(os.path.join(os.getcwd(), '../../src'))
from AIzymes_014 import *

# GENERAL SETTINGS FOR AI.zymes -------------------------------------------------------------------------------------------
AIzymes = AIzymes_MAIN()
AIzymes.setup(FOLDER_HOME = 'TEST_RAVEN',
  FOLDER_PARENT     = 'parent',
  MAX_JOBS          = 5,
  N_PARENT_JOBS     = 7,
  MAX_DESIGNS       = 10,
  KBT_BOLTZMANN     = [0.5, 0.0003],
  CST_WEIGHT        = 1.0,
  CST_NAME          = "5TS_enzdes_planar_tAB100",
  WT                = "7vuu",
  LIGAND            = "5TS",
  DESIGN            = "4,8,16,21,25,28,40,41,44,52,57,60,61",
  SYSTEM            = "RAVEN",
  EXPLORE           = True,
  RUN_PARALLEL      = True,
  SUBMIT_PREFIX     = "TEST"
 )

2025-02-19 10:22:39 - INFO - Job will be run in PARALLEL mode with 5 cpus. It is recommended to set N_PARENT_JOBS to at least 10.
2025-02-19 10:22:39 - INFO - Add a chheck here for GPU request, failing if GPUs are requested wrongly!


Do you really want to restart AIzymes from scratch? 
    This will delete all existing files in /raven/u/bunzela/AIzymes/design/TEST/TEST_RAVEN [y/n]

     y


2025-02-19 10:22:41 - INFO - Running AI.zymes setup.
2025-02-19 10:22:41 - INFO - Content of /raven/u/bunzela/AIzymes/design/TEST/TEST_RAVEN deleted.
2025-02-19 10:22:41 - INFO - Happy AI.zymeing! :)


AIzymes initiated.


In [2]:
import sys, os
if os.path.join(os.getcwd(), '../../src') not in sys.path: sys.path.append(os.path.join(os.getcwd(), '../../src'))
from AIzymes_014 import *
AIzymes = AIzymes_MAIN()
AIzymes.initialize(FOLDER_HOME = 'TEST_RAVEN', LOG="debug", PRINT_VAR=False) 
AIzymes.submit_controller()

2025-02-19 10:22:42 - INFO - Starting parallel controller.


In [18]:
!squeue --me  
#!ls TEST_RAVEN		
#!scancel --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [19]:
!tail -n 20 TEST_RAVEN/logfile.log

2025-02-19 10:27:34 - DEBUG - Updating scores
2025-02-19 10:27:34 - DEBUG - Unblocked ESMfold index 0.
2025-02-19 10:27:34 - DEBUG - Unblocked ESMfold index 3.
2025-02-19 10:27:34 - DEBUG - Unblocked ESMfold index 4.
2025-02-19 10:27:34 - DEBUG - Boltzmann selection tries to select a variant for design, but all are blocked. Waiting 20 seconds
2025-02-19 10:27:54 - DEBUG - 1 parallel jobs.
2025-02-19 10:27:54 - DEBUG - Updating scores
2025-02-19 10:27:54 - DEBUG - Unblocked ESMfold index 6.
2025-02-19 10:27:54 - DEBUG - Boltzmann selection tries to select a variant for design, but all are blocked. Waiting 20 seconds
2025-02-19 10:28:14 - DEBUG - 1 parallel jobs.
2025-02-19 10:28:14 - DEBUG - Updating scores
2025-02-19 10:28:14 - DEBUG - Boltzmann selection tries to select a variant for design, but all are blocked. Waiting 20 seconds
2025-02-19 10:28:34 - DEBUG - 0 parallel jobs.
2025-02-19 10:28:34 - DEBUG - Updating scores
2025-02-19 10:28:34 - DEBUG - Unblocked RosettaRelax index 5.
2

In [20]:
!tail -n 50 TEST_RAVEN/controller.log

2025-02-19 10:28:14 - DEBUG - Updating scores
2025-02-19 10:28:14 - DEBUG - Boltzmann selection tries to select a variant for design, but all are blocked. Waiting 20 seconds
2025-02-19 10:28:34 - DEBUG - 0 parallel jobs.
2025-02-19 10:28:34 - DEBUG - Updating scores
2025-02-19 10:28:34 - DEBUG - Unblocked RosettaRelax index 5.
2025-02-19 10:28:34 - DEBUG - Unblocked ESMfold index 5.
2025-02-19 10:28:34 - INFO - Updated scores and potentials of index 5.
2025-02-19 10:28:34 - INFO - 5 selected because its relaxed but nothing was designed from it.
2025-02-19 10:28:34 - DEBUG - Starting new calculation for index 5.
2025-02-19 10:28:34 - DEBUG - Child index 7 created for 5.
Unblocking all
Traceback (most recent call last):
  File "/raven/u/bunzela/AIzymes/design/TEST/TEST_RAVEN/start_controller_parallel.py", line 9, in <module>
    AIzymes.controller()
  File "/raven/u/bunzela/AIzymes/design/TEST/../../src/AIzymes_014.py", line 170, in controller
    start_controller(self)
  File "/raven/u/

In [17]:
!ls TEST_RAVEN		

7vuu_with_X_as_wildecard.seq  logfile.log	  start_controller_parallel.py
all_scores.csv		      n_running_jobs.dat  submit_controller_parallel.sh
controller.log		      parent		  variables.json
cst.dat			      plots
